In [1]:
import re
import string
import json
import os
import shutil
from pprint import pprint
from functools import partial
from operator import itemgetter, attrgetter
from os.path import normpath
from typing import Callable

import joblib
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns

import dask
import dask.dataframe as dd

# Set Seaborn theme and default palette
sns.set_theme(font_scale=1.25, style="darkgrid")
sns.set_palette("deep", desat=0.85, color_codes=True)

# Turn on inline plotting
%matplotlib inline

# Load Black auto-formatter
%load_ext nb_black

# Enable automatic reloading
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
# Import my modules
from tools import cleaning, plotting, language as lang, outliers, utils
from tools.sklearn.vectorizers import FreqVectorizer, VaderVectorizer
from tools.sklearn.classification import diagnostics as diag
from tools.sklearn import selection

# Run time-consuming grid searches
RUN_SWEEPS = False

# Set my default MPL settings
plt.rcParams.update(plotting.MPL_DEFAULTS)

# dask.config.set(scheduler="threading")

<IPython.core.display.Javascript object>

# Drop Nulls

In [3]:
df = dd.read_parquet("data/amazon", engine="pyarrow")
df

,category,description,title,brand,feature,main_cat,price,asin
npartitions=151,,,,,,,,
0,object,object,object,object,object,object,object,object
100000,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
15000000,...,...,...,...,...,...,...,...
15023058,...,...,...,...,...,...,...,...


<IPython.core.display.Javascript object>

In [4]:
df["description"] = df.loc[:, "description"].str.join(" ").str.strip()
df

,category,description,title,brand,feature,main_cat,price,asin
npartitions=151,,,,,,,,
0,object,object,object,object,object,object,object,object
100000,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
15000000,...,...,...,...,...,...,...,...
15023058,...,...,...,...,...,...,...,...


<IPython.core.display.Javascript object>

In [5]:
df = df.mask(df.applymap(len) == 0)
df

,category,description,title,brand,feature,main_cat,price,asin
npartitions=151,,,,,,,,
0,object,object,object,object,object,object,object,object
100000,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
15000000,...,...,...,...,...,...,...,...
15023058,...,...,...,...,...,...,...,...


<IPython.core.display.Javascript object>

In [6]:
df = df.dropna(
    subset=[
        "category",
        "description",
        "title",
        "price",
        "main_cat",
        "asin",
    ]
)
df

,category,description,title,brand,feature,main_cat,price,asin
npartitions=151,,,,,,,,
0,object,object,object,object,object,object,object,object
100000,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
15000000,...,...,...,...,...,...,...,...
15023058,...,...,...,...,...,...,...,...


<IPython.core.display.Javascript object>

In [7]:
df = df.compute()
df.shape

(5540229, 8)

<IPython.core.display.Javascript object>

In [8]:
df.head()

,category,description,title,brand,feature,main_cat,price,asin
0,"[Clothing, Shoes & Jewelry, Women, Clothing, T...",(=^ ^=) 1.It is made of high quality materials...,"Women Blouse, Ninasill Hooded Sweatshirt Coat ...",Ninasill_Blouse,"[Import, Versatile Occasions - Great for Daily...",Movies & TV,$9.99 - $12.50,6305121869
1,"[Clothing, Shoes & Jewelry, Traditional & Cult...","<b><br>Hi friend, welcome to our store. Before...",Coolred-Women Retro Split Sexy Skinny Premium ...,Coolred-Women,"[100% Spandex, Order Larger If You are Between...","<img src=""https://images-na.ssl-images-amazon....",$16.08 - $18.36,6318708057
2,"[Clothing, Shoes & Jewelry, Men, Clothing, Sho...",Gaok men's pants are all made from cotton mate...,Gaok Men's Retro Cotton Multi-Pocket Camo Carg...,Gaok,"[Cotton, Zipper closure, Material:100% cotton,...","<img src=""https://images-na.ssl-images-amazon....",$26.99,6342506256
3,"[Clothing, Shoes & Jewelry, Men, Clothing, Jea...",<b>pant size(Unit:inch)</b><br> W30(tag30) Wai...,QIBOE Men's Baggy Jeans Denim Sweatpants Loose...,QIBOE,"[Denim, Zipper closure, Material: cotton, Styl...","<img src=""https://images-na.ssl-images-amazon....",$38.99,6342509379
4,"[Clothing, Shoes & Jewelry, Women, Accessories...",All of our sunglasses offer protection against...,Crazy Explosion-proof Lens Polarized Sunglasse...,OLO,"[Material: Polycarbonate, Main features: Windp...",Sports & Outdoors,$3.29,6342522081


<IPython.core.display.Javascript object>

# Dedupe

In [9]:
df = df.drop_duplicates(subset=["asin"])
df.shape

(5422140, 8)

<IPython.core.display.Javascript object>

In [10]:
df = df.drop_duplicates(subset=["title", "brand"])
df.shape

(5275223, 8)

<IPython.core.display.Javascript object>

In [11]:
cleaning.dup_rows(df, subset="description")

,category,description,title,brand,feature,main_cat,price,asin
4,"[Clothing, Shoes & Jewelry, Women, Accessories...",All of our sunglasses offer protection against...,Crazy Explosion-proof Lens Polarized Sunglasse...,OLO,"[Material: Polycarbonate, Main features: Windp...",Sports & Outdoors,$3.29,6342522081
6,"[Clothing, Shoes & Jewelry, Women, Accessories...",All of our sunglasses offer protection against...,Crazy High Explosion-proof Profile Sport Cycli...,OLO,"[Material: Polycarbonate, Main features: Windp...",Sports & Outdoors,$3.99,6342522235
8,"[Clothing, Shoes & Jewelry, Women, Clothing, L...",Material : Core-spun fabric silk <br> feature:...,FQQ Women Sexy Lingerie Lace Dress Sheer Babyd...,FQQ,"[100% Polyester, We use comfortable tissue to ...","<img src=""https://images-na.ssl-images-amazon....",$2.80,6342522545
10,"[Clothing, Shoes & Jewelry, Women, Clothing, D...",Material : Core-spun fabric silk <br> feature:...,Crazy Women's Sexy Leather Backless Bodycon Cl...,Crazy,"[100% Polyester, We use comfortable tissue to ...","<img src=""https://images-na.ssl-images-amazon....",$8.50,6342522898
11,"[Clothing, Shoes & Jewelry, Women, Clothing, L...",Material : Core-spun fabric silk <br> feature:...,FQQ Women's Sexy Lingerie Babydoll Dress Sleep...,FQQ,"[100% Polyester, We use comfortable tissue to ...","<img src=""https://images-na.ssl-images-amazon....",$4.50,6342523002
...,...,...,...,...,...,...,...,...
15009750,"[Software, Education & Reference, Maps & Atlases]",Featuring exclusive birdseye satellite imagery...,Garmin Huntview Map Card - Pennsylvania,Garmin,[Topo 24K - digital topography equivalent to 1...,Cell Phones & Accessories,$74.95,B01FWPMENI
15009751,"[Software, Education & Reference, Maps & Atlases]",Featuring exclusive birdseye satellite imagery...,Garmin Huntview Map Card - Wisconsin,Garmin,[Topo 24K - digital topography equivalent to 1...,Cell Phones & Accessories,$74.95,B01FWPMJLU
15009752,"[Software, Education & Reference, Maps & Atlases]",Featuring exclusive birdseye satellite imagery...,Garmin 010-12511-00 Huntview Map Card - Michigan,Garmin,[Topo 24K - digital topography equivalent to 1...,Cell Phones & Accessories,$74.95,B01FWPMEWO
15009822,"[Software, Children's]",<b>Features: </b><br>1. Beautiful and fabulous...,925 Sterling Silver Angel Wings Heart Pendant ...,17maimeng,[Material: Guaranteed 100% Real Silver+Platinu...,"<img src=""https://images-na.ssl-images-amazon....",$24.00,B01HEFZJC2


<IPython.core.display.Javascript object>

In [12]:
df = df.drop_duplicates(subset=["description"])
df.shape

(4179260, 8)

<IPython.core.display.Javascript object>

In [13]:
df.reset_index(drop=True, inplace=True)
df.tail()

,category,description,title,brand,feature,main_cat,price,asin
4179255,"[Software, Photography]",Make movies to impress with the hands-on editi...,Pinnacle Studio 20 (Old Version),Pinnacle Systems,"[New - Stop Motion Animation, Enhanced - Multi...",Software,$27.99,B01HAP3TX2
4179256,"[Software, Photography]",CorelDRAW Home & Student Suite X8 is an afford...,CorelDRAW Home &amp; Student Suite X8 for PC (...,Corel,[Produce high-quality photos and graphics with...,Software,$67.48,B01HAP3NJM
4179257,"[Software, Children's, Early Learning]",<b>Features: </b><br>1. Beautiful and fabulous...,925 Sterling Silver Purple Crystal Star Drop E...,17maimeng,[Material: Guaranteed 100% Real Silver+Platinu...,"<img src=""https://images-na.ssl-images-amazon....",$24.40,B01HCRZM4C
4179258,"[Software, Children's]",<b>Features: </b><br>1. Beautiful and fabulous...,925 Sterling Silver Angel Wings Heart Pendant ...,17maimeng,[Material: Guaranteed 100% Real Silver+Platinu...,"<img src=""https://images-na.ssl-images-amazon....",$24.00,B01HEFZJC2
4179259,"[Software, Digital Software, Antivirus & Secur...",<div>Mac Internet Security X9 contains two of ...,Intego Mac Internet Security X9 - 1 Mac - 1 ye...,Intego,[Award-winning antivirus software to protect a...,Software,$39.99,B01HF3G4BS


<IPython.core.display.Javascript object>

# Clean Main Category

In [14]:
is_html = df["main_cat"].str.match(r"^<\w+")
extracted_cats = df.loc[is_html, "main_cat"].str.extract(r"alt=\"(.+)\"")
df["main_cat"].update(extracted_cats.squeeze())
df["main_cat"].unique()

array(['Movies & TV', 'AMAZON FASHION', 'Sports & Outdoors',
       'Amazon Home', 'Health & Personal Care', 'Baby', 'Office Products',
       'Digital Music', 'Video Games', 'Toys & Games',
       'Tools & Home Improvement', 'All Beauty', 'All Electronics',
       'Industrial & Scientific', 'Camera & Photo', 'Automotive',
       'Computers', 'Arts, Crafts & Sewing', 'Car Electronics',
       'Musical Instruments', 'Software', 'Books', 'Pet Supplies',
       'Grocery', 'Appliances', 'Home Audio & Theater', 'Luxury Beauty',
       'Cell Phones & Accessories', 'Portable Audio & Accessories',
       'Gift Cards', 'MEMBERSHIPS & SUBSCRIPTIONS', 'Amazon Devices',
       'Health &amp; Personal Care', 'Arts, Crafts &amp; Sewing',
       'Toys &amp; Games', 'Industrial &amp; Scientific',
       'Tools &amp; Home Improvement', 'Sports &amp; Outdoors',
       'Camera &amp; Photo', 'Home Audio &amp; Theater',
       'Portable Audio &amp; Accessories',
       'Cell Phones &amp; Accessories', 'GPS 

<IPython.core.display.Javascript object>

In [15]:
df["main_cat"] = lang.decode_html_entities(df["main_cat"])
df["main_cat"] = df["main_cat"].str.lower()
df["main_cat"].unique()

array(['movies & tv', 'amazon fashion', 'sports & outdoors',
       'amazon home', 'health & personal care', 'baby', 'office products',
       'digital music', 'video games', 'toys & games',
       'tools & home improvement', 'all beauty', 'all electronics',
       'industrial & scientific', 'camera & photo', 'automotive',
       'computers', 'arts, crafts & sewing', 'car electronics',
       'musical instruments', 'software', 'books', 'pet supplies',
       'grocery', 'appliances', 'home audio & theater', 'luxury beauty',
       'cell phones & accessories', 'portable audio & accessories',
       'gift cards', 'memberships & subscriptions', 'amazon devices',
       'gps & navigation', 'apple products', 'collectibles & fine art',
       'sports collectibles', 'handmade', 'prime pantry',
       'amazon fire tv', 'magazine subscriptions', 'collectible coins'],
      dtype=object)

<IPython.core.display.Javascript object>

In [16]:
main_cat_counts = df["main_cat"].value_counts()
main_cat_counts.to_frame().style.bar()

,main_cat
books,911658
amazon home,555380
amazon fashion,488814
automotive,310766
sports & outdoors,272053
toys & games,244481
tools & home improvement,234666
digital music,177178
office products,103549
industrial & scientific,103162


<IPython.core.display.Javascript object>

In [17]:
keep = set(main_cat_counts.loc[main_cat_counts >= 100].index)
df = df.loc[df["main_cat"].isin(keep)]
df["main_cat"].value_counts()

books                           911658
amazon home                     555380
amazon fashion                  488814
automotive                      310766
sports & outdoors               272053
toys & games                    244481
tools & home improvement        234666
digital music                   177178
office products                 103549
industrial & scientific         103162
grocery                         100899
arts, crafts & sewing            99708
movies & tv                      92855
cell phones & accessories        74771
computers                        70788
pet supplies                     63699
all electronics                  48115
musical instruments              47368
camera & photo                   45033
home audio & theater             38438
video games                      36118
health & personal care           19560
all beauty                        9441
car electronics                   9316
baby                              8919
software                 

<IPython.core.display.Javascript object>

In [18]:
cats = df["category"].explode()
cats = lang.decode_html_entities(cats)
cats = cats.str.lower().str.strip()
cats

0             clothing, shoes & jewelry
0                                 women
0                              clothing
0                  tops, tees & blouses
0          blouses & button-down shirts
                       ...             
4179258                      children's
4179259                        software
4179259                digital software
4179259            antivirus & security
4179259               internet security
Name: category, Length: 17965683, dtype: object

<IPython.core.display.Javascript object>

In [19]:
cats.value_counts().to_frame("Count")

,Count
books,911026
"clothing, shoes & jewelry",521061
home & kitchen,422425
automotive,311592
women,279972
...,...
lovely fly ruffled cuff,1
made in the usa by night owl jewelry from materials sourced in the usa!,1
alfa global women's plus size abstract printed harem pants with 2 real front pockets,1
delivery time: about 3 weeks - boys kids saudi style thobe thoub robe daffah dishdasha islamic arabian kaftan,1


<IPython.core.display.Javascript object>

In [20]:
cats = cats.to_frame()
cats["n_char"] = cats["category"].map(len)
cats.index.name = "product_idx"
cats = cats.reset_index()
cats

,product_idx,category,n_char
0,0,"clothing, shoes & jewelry",25
1,0,women,5
2,0,clothing,8
3,0,"tops, tees & blouses",20
4,0,blouses & button-down shirts,28
...,...,...,...
17965678,4179258,children's,10
17965679,4179259,software,8
17965680,4179259,digital software,16
17965681,4179259,antivirus & security,20


<IPython.core.display.Javascript object>

In [21]:
cats = cats.loc[outliers.tukey_trim(cats["n_char"]).index]
cats

,n_trimmed,pct_trimmed
n_char,564936,3.144528
total_obs,564936,3.144528


,product_idx,category,n_char
0,0,"clothing, shoes & jewelry",25
1,0,women,5
2,0,clothing,8
3,0,"tops, tees & blouses",20
4,0,blouses & button-down shirts,28
...,...,...,...
17965678,4179258,children's,10
17965679,4179259,software,8
17965680,4179259,digital software,16
17965681,4179259,antivirus & security,20


<IPython.core.display.Javascript object>

In [22]:
forbidden_punct = string.punctuation.replace(",", "").replace("&", "")
forbidden_punct

'!"#$%\'()*+-./:;<=>?@[\\]^_`{|}~'

<IPython.core.display.Javascript object>

I remove categories containing digits or forbidden punctuation symbols.

In [23]:
cats = cats.loc[~cats.category.str.contains(f"[\d{forbidden_punct}]")]
cats

,product_idx,category,n_char
1,0,women,5
2,0,clothing,8
5,0,import,6
14,1,traditional & cultural wear,27
15,1,asian,5
...,...,...,...
17965677,4179258,software,8
17965679,4179259,software,8
17965680,4179259,digital software,16
17965681,4179259,antivirus & security,20


<IPython.core.display.Javascript object>

In [24]:
cat_counts = cats.category.value_counts().reset_index()
cat_counts = cat_counts.rename(
    columns={
        "index": "category",
        "category": "count",
    }
)
cat_counts

,category,count
0,books,911026
1,home & kitchen,422425
2,automotive,311592
3,women,279972
4,sports & outdoors,273973
...,...,...
126030,polished yellow and rose ip plated,1
126031,adorable striped swimsuit set,1
126032,lightly cushioned collar,1
126033,woven top handles,1


<IPython.core.display.Javascript object>

In [25]:
cats = pd.merge(cats, cat_counts, on="category", how="left")
cats

,product_idx,category,n_char,count
0,0,women,5,279972
1,0,clothing,8,228479
2,0,import,6,240
3,1,traditional & cultural wear,27,945
4,1,asian,5,1491
...,...,...,...,...
15195925,4179258,software,8,5714
15195926,4179259,software,8,5714
15195927,4179259,digital software,16,742
15195928,4179259,antivirus & security,20,238


<IPython.core.display.Javascript object>

In [26]:
cats = cats.loc[cats["count"] >= 100]
cats["category"].value_counts()

books                      911026
home & kitchen             422425
automotive                 311592
women                      279972
sports & outdoors          273973
                            ...  
pickleball                    100
hypnosis                      100
sledgehammers                 100
complimentary packaging       100
gypsy                         100
Name: category, Length: 6102, dtype: int64

<IPython.core.display.Javascript object>

In [27]:
cats = cats.groupby("product_idx")["category"].agg(lambda x: x.to_list())
cats = cats.reindex_like(df)
cats

0                                  [women, clothing, import]
1                       [traditional & cultural wear, asian]
2          [men, clothing, shorts, cargo, cotton, zipper ...
3              [men, clothing, jeans, denim, zipper closure]
4          [women, accessories, sunglasses & eyewear acce...
                                 ...                        
4179255                              [software, photography]
4179256                              [software, photography]
4179257                           [software, early learning]
4179258                                           [software]
4179259    [software, digital software, antivirus & secur...
Name: category, Length: 4179222, dtype: object

<IPython.core.display.Javascript object>

In [28]:
df["category"] = cats
df.head()

,category,description,title,brand,feature,main_cat,price,asin
0,"[women, clothing, import]",(=^ ^=) 1.It is made of high quality materials...,"Women Blouse, Ninasill Hooded Sweatshirt Coat ...",Ninasill_Blouse,"[Import, Versatile Occasions - Great for Daily...",movies & tv,$9.99 - $12.50,6305121869
1,"[traditional & cultural wear, asian]","<b><br>Hi friend, welcome to our store. Before...",Coolred-Women Retro Split Sexy Skinny Premium ...,Coolred-Women,"[100% Spandex, Order Larger If You are Between...",amazon fashion,$16.08 - $18.36,6318708057
2,"[men, clothing, shorts, cargo, cotton, zipper ...",Gaok men's pants are all made from cotton mate...,Gaok Men's Retro Cotton Multi-Pocket Camo Carg...,Gaok,"[Cotton, Zipper closure, Material:100% cotton,...",amazon fashion,$26.99,6342506256
3,"[men, clothing, jeans, denim, zipper closure]",<b>pant size(Unit:inch)</b><br> W30(tag30) Wai...,QIBOE Men's Baggy Jeans Denim Sweatpants Loose...,QIBOE,"[Denim, Zipper closure, Material: cotton, Styl...",amazon fashion,$38.99,6342509379
4,"[women, accessories, sunglasses & eyewear acce...",All of our sunglasses offer protection against...,Crazy Explosion-proof Lens Polarized Sunglasse...,OLO,"[Material: Polycarbonate, Main features: Windp...",sports & outdoors,$3.29,6342522081


<IPython.core.display.Javascript object>

In [29]:
del cats

<IPython.core.display.Javascript object>

In [30]:
df.isna().sum()

category          1800
description          0
title                0
brand           148378
feature        1546141
main_cat             0
price                0
asin                 0
dtype: int64

<IPython.core.display.Javascript object>

In [31]:
df = df.dropna(subset=["category"])
df.isna().sum()

category             0
description          0
title                0
brand           148375
feature        1545999
main_cat             0
price                0
asin                 0
dtype: int64

<IPython.core.display.Javascript object>

# Clean Price

In [32]:
df["price"] = df.loc[:, "price"].str.extract(r"\$?(\d+\.\d{2})")
df["price"] = df.loc[:, "price"].astype(np.float64)
df.head(10)

,category,description,title,brand,feature,main_cat,price,asin
0,"[women, clothing, import]",(=^ ^=) 1.It is made of high quality materials...,"Women Blouse, Ninasill Hooded Sweatshirt Coat ...",Ninasill_Blouse,"[Import, Versatile Occasions - Great for Daily...",movies & tv,9.99,6305121869
1,"[traditional & cultural wear, asian]","<b><br>Hi friend, welcome to our store. Before...",Coolred-Women Retro Split Sexy Skinny Premium ...,Coolred-Women,"[100% Spandex, Order Larger If You are Between...",amazon fashion,16.08,6318708057
2,"[men, clothing, shorts, cargo, cotton, zipper ...",Gaok men's pants are all made from cotton mate...,Gaok Men's Retro Cotton Multi-Pocket Camo Carg...,Gaok,"[Cotton, Zipper closure, Material:100% cotton,...",amazon fashion,26.99,6342506256
3,"[men, clothing, jeans, denim, zipper closure]",<b>pant size(Unit:inch)</b><br> W30(tag30) Wai...,QIBOE Men's Baggy Jeans Denim Sweatpants Loose...,QIBOE,"[Denim, Zipper closure, Material: cotton, Styl...",amazon fashion,38.99,6342509379
4,"[women, accessories, sunglasses & eyewear acce...",All of our sunglasses offer protection against...,Crazy Explosion-proof Lens Polarized Sunglasse...,OLO,"[Material: Polycarbonate, Main features: Windp...",sports & outdoors,3.29,6342522081
5,"[women, clothing, leggings]",Y-BIN Women's Leggings Fitted Capris Gym Worko...,Y-BIN Women's Capri Tights leggings Yoga Pants...,Y-BIN,"[cotton blended, Has a fashionable flap over P...",amazon fashion,7.99,6342520577
6,"[women, accessories, scarves & wraps, fashion ...",Feature <br> -Great quality winter scarf. <br>...,Crazy Women's Voile Crinkle Scarf Shawl,Crazy,"[Material:voile, Size: 180cm x 95cm, A scarf t...",amazon fashion,0.50,6342502315
7,"[women, clothing, sleep & lounge, nightgowns &...",Material : Core-spun fabric silk <br> feature:...,FQQ Women Sexy Lingerie Lace Dress Sheer Babyd...,FQQ,"[100% Polyester, We use comfortable tissue to ...",amazon fashion,2.80,6342522545
8,"[novelty & more, clothing, novelty, men, shirts]",A provocative woody spicy masculine scent that...,Armaf Club De Nuit Intense 3.4 Eau De Toilette...,F ARMAF,NaN,amazon home,29.74,6338750458
9,"[women, accessories, scarves & wraps, wraps & ...",Feature<br> -Great quality mens winter scarf.<...,CRAZY Elegant Jacquard Weave Pashmina Shawl Wr...,Crazy,"[100% Acrylic, Acrylic/Artificial wool, Size: ...",amazon fashion,19.90,6342501416


<IPython.core.display.Javascript object>

In [33]:
display(df.isna().sum())
df.shape

category             0
description          0
title                0
brand           148375
feature        1545999
main_cat             0
price            78365
asin                 0
dtype: int64

(4177422, 8)

<IPython.core.display.Javascript object>

# Clean Brand

In [34]:
df["brand"] = df["brand"].map(lang.decode_html_entities, "ignore")
df["brand"] = df["brand"].str.lower()
df["brand"] = df["brand"].map(lang.strip_multiwhite, "ignore")
df["brand"].head()

0    ninasill_blouse
1      coolred-women
2               gaok
3              qiboe
4                olo
Name: brand, dtype: object

<IPython.core.display.Javascript object>

In [35]:
brand_counts = df["brand"].value_counts()
brand_counts

uxcell                                 15216
3drose                                 12727
unknown                                11027
generic                                 9725
various artists                         8172
                                       ...  
visit amazon's caitlyn schmitt page        1
norma ricketts                             1
dr. roger w sapp                           1
suzy frank/linda lara                      1
cat clarke                                 1
Name: brand, Length: 881870, dtype: int64

<IPython.core.display.Javascript object>

In [36]:
keep = brand_counts.loc[brand_counts >= 100].copy()
keep.drop(["unknown", "generic", "n/a", "not available", "none"], inplace=True)
keep = set(keep.index)
keep

{'4keyboard',
 'sincerely nuts',
 'latchwell',
 'fengshuisale',
 'hairbow center llc',
 'ryobi',
 'trlyc',
 'mountain warehouse',
 'north american bear',
 'ist',
 'texas instruments',
 'packard',
 'sublime gifts',
 'imprue',
 'sunbelt radiators',
 'aminco',
 'mg posters',
 'camco',
 'mr. bar-b-q',
 'forever',
 'fakeface',
 'tamiya',
 'things2die4',
 'baumgartens',
 'trident',
 'stancemagic',
 'mahamosa gourmet teas, spices & herbs',
 'truglo',
 'the prepared pantry',
 'shidianyi',
 'smartsign',
 'general mills',
 'apache',
 'heart of america',
 'trimax',
 'lion brand yarn',
 'mariell',
 'lemonbest',
 'lawn fawn',
 'videosecu',
 'shirtloco',
 'aven',
 'pacific giftware',
 'football fanatics',
 'ravensburger',
 'kidkraft',
 'starbucks',
 'gore wear',
 'supermicro',
 "k'nex",
 'diften',
 'pomona',
 'dwk',
 'vidpro',
 'ridgid',
 'bostitch',
 'anchor hocking',
 'cwi gifts',
 'nobleworks',
 'wild republic',
 'gundam',
 'littlearth',
 'pro force',
 'bruder toys',
 'rapid dominance',
 'the ice

<IPython.core.display.Javascript object>

In [37]:
df.loc[~df.brand.isin(keep), "brand"] = np.nan
df.brand.sample(100, random_state=35)

277000                   NaN
815248                   NaN
282635     in season jewelry
3292577                  NaN
4004925               uxcell
                 ...        
1134061                  NaN
2610041           posterenvy
2157914           duragadget
1167074                  NaN
2947999                  NaN
Name: brand, Length: 100, dtype: object

<IPython.core.display.Javascript object>

In [38]:
visit_amazon = df.brand.str.startswith("visit amazon's") == True
df.loc[visit_amazon]

,category,description,title,brand,feature,main_cat,price,asin
938152,"[books, literature & fiction, dramas & plays]",William Shakespeare is widely regarded as the ...,Love's Labour's Lost: Performed by Derek Jacob...,visit amazon's william shakespeare page,NaN,books,20.93,0001050230
938257,"[books, literature & fiction, genre fiction]","With more than 120 titles still in print, Loui...",The Collected Short Stories of Louis L'Amour -...,visit amazon's louis l'amour page,NaN,books,102.48,0002103621
938282,"[books, literature & fiction, genre fiction]",The collected short stories of Louis L'Amour t...,The Collected Short Stories of Louis L'Amour -...,visit amazon's louis l'amour page,NaN,books,231.38,0002103648
938406,"[books, thrillers & suspense]","""Casanova"" works the East Coast, ""The Gentlema...",Kiss The Girls,visit amazon's james patterson page,NaN,books,6.80,0002241277
938429,"[books, literature & fiction, genre fiction]",A child killer is stalking the inner city of W...,Jack And Jill,visit amazon's james patterson page,NaN,books,6.93,0002252317
...,...,...,...,...,...,...,...,...
2510983,"[office products, office & school supplies, bo...","Made from durable nylon material, this sporty ...",Compass Med Book and Bible Cover,visit amazon's zondervan page,NaN,books,16.25,0310806607
2510984,"[office products, office & school supplies, bo...",This multi-functional organizers features a de...,Bible Study Organizer Coal with Leather-Look A...,visit amazon's zondervan page,NaN,books,23.38,0310818605
2510985,"[office products, office & school supplies, bo...","Featuring metal accents, purse-style handles a...",Reptile Leather Extra Large Wine Bible Cover,visit amazon's zondervan page,NaN,books,13.95,0310821800
2510986,"[office products, office & school supplies, bo...","Trendy, bright flower design for spring---sure...",Daisy Microfiber Pink Book &amp; Bible Cover,visit amazon's zondervan page,NaN,books,12.95,0310822254


<IPython.core.display.Javascript object>

In [45]:
pages = df.loc[visit_amazon, "brand"].str.extract(r"visit amazon's (.+) page")
pages

,0
938152,NaN
938257,NaN
938282,NaN
938406,NaN
938429,NaN
...,...
2510983,NaN
2510984,NaN
2510985,NaN
2510986,NaN


<IPython.core.display.Javascript object>

In [44]:
df.loc[visit_amazon, "brand"] = pages.squeeze()
df.loc[visit_amazon]

,category,description,title,brand,feature,main_cat,price,asin
938152,"[books, literature & fiction, dramas & plays]",William Shakespeare is widely regarded as the ...,Love's Labour's Lost: Performed by Derek Jacob...,william shakespeare,NaN,books,20.93,0001050230
938257,"[books, literature & fiction, genre fiction]","With more than 120 titles still in print, Loui...",The Collected Short Stories of Louis L'Amour -...,louis l'amour,NaN,books,102.48,0002103621
938282,"[books, literature & fiction, genre fiction]",The collected short stories of Louis L'Amour t...,The Collected Short Stories of Louis L'Amour -...,louis l'amour,NaN,books,231.38,0002103648
938406,"[books, thrillers & suspense]","""Casanova"" works the East Coast, ""The Gentlema...",Kiss The Girls,james patterson,NaN,books,6.80,0002241277
938429,"[books, literature & fiction, genre fiction]",A child killer is stalking the inner city of W...,Jack And Jill,james patterson,NaN,books,6.93,0002252317
...,...,...,...,...,...,...,...,...
2510983,"[office products, office & school supplies, bo...","Made from durable nylon material, this sporty ...",Compass Med Book and Bible Cover,zondervan,NaN,books,16.25,0310806607
2510984,"[office products, office & school supplies, bo...",This multi-functional organizers features a de...,Bible Study Organizer Coal with Leather-Look A...,zondervan,NaN,books,23.38,0310818605
2510985,"[office products, office & school supplies, bo...","Featuring metal accents, purse-style handles a...",Reptile Leather Extra Large Wine Bible Cover,zondervan,NaN,books,13.95,0310821800
2510986,"[office products, office & school supplies, bo...","Trendy, bright flower design for spring---sure...",Daisy Microfiber Pink Book &amp; Bible Cover,zondervan,NaN,books,12.95,0310822254


<IPython.core.display.Javascript object>

In [126]:
# null_brand = cleaning.null_rows(df, subset="brand")
# null_brand

<IPython.core.display.Javascript object>

In [124]:
# search_brands = brand_counts.loc[keep].sort_values(0, False).index.to_series()
# search_brands.drop(["various", "various artists"], inplace=True)
# search_brands.reset_index(drop=True, inplace=True)
# search_brands = search_brands.loc[search_brands.str.len() > 5]
# search_brands

<IPython.core.display.Javascript object>

In [125]:
# lang.locate_patterns(
#     *search_brands.head(10),
#     strings=null_brand.title,
#     flags=re.I,
#     exclusive=True,
# )

<IPython.core.display.Javascript object>

# Export

In [127]:
path = "data/amazon_clean"
if os.path.isdir(path):
    shutil.rmtree(path)

df.to_parquet(
    path,
    engine="pyarrow",
    index=None,
    partition_cols=["main_cat"],
)

<IPython.core.display.Javascript object>